In [3]:
import pandas as pd

# Replace with the actual path to your CSV
csv_file = "model_states.csv"

# Read the CSV into a pandas DataFrame
model_state = pd.read_csv(csv_file)

# See the first few rows
print(model_state.head())

# Check the column names
print(model_state.columns)

# (Optional) Some basic stats
print(model_state.describe())

model_state


         %time  field.name0 field.name1                field.name2  \
0  14009000000  ocean_waves      vessel  120m_landscape_smooth_tri   
1  14009000000  ocean_waves      vessel  120m_landscape_smooth_tri   
2  14009000000  ocean_waves      vessel  120m_landscape_smooth_tri   
3  14009000000  ocean_waves      vessel  120m_landscape_smooth_tri   
4  14009000000  ocean_waves      vessel  120m_landscape_smooth_tri   

  field.name3  field.pose0.position.x  field.pose0.position.y  \
0   quadrotor                       0                       0   
1   quadrotor                       0                       0   
2   quadrotor                       0                       0   
3   quadrotor                       0                       0   
4   quadrotor                       0                       0   

   field.pose0.position.z  field.pose0.orientation.x  \
0                       0                          0   
1                       0                          0   
2                   

,%time,field.name0,field.name1,field.name2,field.name3,field.pose0.position.x,field.pose0.position.y,field.pose0.position.z,field.pose0.orientation.x,field.pose0.orientation.y,...,field.twist2.linear.z,field.twist2.angular.x,field.twist2.angular.y,field.twist2.angular.z,field.twist3.linear.x,field.twist3.linear.y,field.twist3.linear.z,field.twist3.angular.x,field.twist3.angular.y,field.twist3.angular.z
0,14009000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0,0.312384,-0.320109,0.041819,-0.450783,-0.042231,0.005638
1,14009000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0,0.312283,-0.322007,0.041940,-0.450642,-0.043740,0.005333
2,14009000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0,0.312182,-0.323900,0.042061,-0.450501,-0.045246,0.005029
3,14009000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0,0.312080,-0.325788,0.042184,-0.450361,-0.046748,0.004728
4,14009000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0,0.311978,-0.327672,0.042308,-0.450221,-0.048247,0.004428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73344,87348000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0,0.707793,0.575176,0.000601,0.015748,-0.032976,-0.000243
73345,87349000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0,0.707667,0.575211,0.000599,0.011504,-0.028567,-0.000310
73346,87350000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0,0.707541,0.575246,0.000596,0.007261,-0.024159,-0.000378
73347,87351000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0,0.707414,0.575281,0.000595,0.011908,-0.028959,-0.000304


In [5]:
# List of columns to keep:
subset_columns = [
    '%time',                  # Timestamp column
    'field.name1',            # Name for the model at index 1
    'field.name3',            # Name for the model at index 3
    # Pose data for model 1:
    'field.pose1.position.x',
    'field.pose1.position.y',
    'field.pose1.position.z',
    'field.pose1.orientation.x',
    'field.pose1.orientation.y',
    'field.pose1.orientation.z',
    # Pose data for model 3:
    'field.pose3.position.x',
    'field.pose3.position.y',
    'field.pose3.position.z',
    'field.pose3.orientation.x',
    'field.pose3.orientation.y',
    'field.pose3.orientation.z'
]

# Create the subset DataFrame
vessel_drone = model_state[subset_columns]

vessel_drone


,%time,field.name1,field.name3,field.pose1.position.x,field.pose1.position.y,field.pose1.position.z,field.pose1.orientation.x,field.pose1.orientation.y,field.pose1.orientation.z,field.pose3.position.x,field.pose3.position.y,field.pose3.position.z,field.pose3.orientation.x,field.pose3.orientation.y,field.pose3.orientation.z
0,14009000000,vessel,quadrotor,-2.550664,-5.272131,2.159016,-0.016152,-0.018357,-0.033576,-2.654839,-4.777739,11.873734,0.094939,-0.004127,-0.003356
1,14009000000,vessel,quadrotor,-2.550484,-5.272079,2.160047,-0.016137,-0.018316,-0.033574,-2.654526,-4.778061,11.873776,0.094715,-0.004149,-0.003351
2,14009000000,vessel,quadrotor,-2.550304,-5.272027,2.161079,-0.016122,-0.018274,-0.033572,-2.654214,-4.778385,11.873818,0.094491,-0.004172,-0.003345
3,14009000000,vessel,quadrotor,-2.550124,-5.271975,2.162112,-0.016107,-0.018233,-0.033570,-2.653902,-4.778710,11.873860,0.094267,-0.004196,-0.003340
4,14009000000,vessel,quadrotor,-2.549945,-5.271923,2.163146,-0.016092,-0.018191,-0.033568,-2.653590,-4.779038,11.873902,0.094043,-0.004220,-0.003334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73344,87348000000,vessel,quadrotor,0.229913,-21.875387,2.753341,-0.010901,0.000716,0.082355,-0.082686,-22.138547,12.000548,-0.003957,-0.003757,0.000124
73345,87349000000,vessel,quadrotor,0.230477,-21.874834,2.753278,-0.010893,0.000697,0.082362,-0.081978,-22.137972,12.000548,-0.003951,-0.003771,0.000124
73346,87350000000,vessel,quadrotor,0.231042,-21.874281,2.753214,-0.010885,0.000679,0.082369,-0.081270,-22.137396,12.000549,-0.003947,-0.003783,0.000124
73347,87351000000,vessel,quadrotor,0.231606,-21.873729,2.753150,-0.010876,0.000661,0.082375,-0.080563,-22.136821,12.000550,-0.003941,-0.003797,0.000123


In [6]:
import numpy as np
# Compute the Euclidean distance between the vessel and the drone
# Adjust the column names if they are different in your file.
model_state['distance'] = np.sqrt(
    (model_state['field.pose3.position.x'] - model_state['field.pose1.position.x'])**2 +
    (model_state['field.pose3.position.y'] - model_state['field.pose1.position.y'])**2 +
    (model_state['field.pose3.position.z'] - model_state['field.pose1.position.z'])**2
)

# Now you can inspect the new 'distance' column
print(model_state[['%time', 'distance']].head())


         %time  distance
0  14009000000  9.727848
1  14009000000  9.726839
2  14009000000  9.725830
3  14009000000  9.724820
4  14009000000  9.723809


In [7]:
model_state

,%time,field.name0,field.name1,field.name2,field.name3,field.pose0.position.x,field.pose0.position.y,field.pose0.position.z,field.pose0.orientation.x,field.pose0.orientation.y,...,field.twist2.angular.x,field.twist2.angular.y,field.twist2.angular.z,field.twist3.linear.x,field.twist3.linear.y,field.twist3.linear.z,field.twist3.angular.x,field.twist3.angular.y,field.twist3.angular.z,distance
0,14009000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0.312384,-0.320109,0.041819,-0.450783,-0.042231,0.005638,9.727848
1,14009000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0.312283,-0.322007,0.041940,-0.450642,-0.043740,0.005333,9.726839
2,14009000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0.312182,-0.323900,0.042061,-0.450501,-0.045246,0.005029,9.725830
3,14009000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0.312080,-0.325788,0.042184,-0.450361,-0.046748,0.004728,9.724820
4,14009000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0.311978,-0.327672,0.042308,-0.450221,-0.048247,0.004428,9.723809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73344,87348000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0.707793,0.575176,0.000601,0.015748,-0.032976,-0.000243,9.256231
73345,87349000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0.707667,0.575211,0.000599,0.011504,-0.028567,-0.000310,9.256289
73346,87350000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0.707541,0.575246,0.000596,0.007261,-0.024159,-0.000378,9.256348
73347,87351000000,ocean_waves,vessel,120m_landscape_smooth_tri,quadrotor,0,0,0,0,0,...,0,0,0,0.707414,0.575281,0.000595,0.011908,-0.028959,-0.000304,9.256407
